In [1]:
import matplotlib
matplotlib.use('Agg')
from pylab import *


import cPickle as pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import pandas as pd
from sklearn import preprocessing
import collections
from sklearn.grid_search import GridSearchCV

In [2]:
kegg_queries = pickle.load( open( "../data/kegg_queries.p", "rb" ) )

In [3]:
keggs = kegg_queries.keys()
#kegg_lists = [str(k) for k in  kegg_queries.values()]
kegg_lists = [str(kegg_queries[kegg]) for kegg in  keggs]

In [4]:
kegg_lists[0]

'ENTRY       K02040                      KO\nNAME        pstS\nDEFINITION  phosphate transport system substrate-binding protein\nPATHWAY     ko02010  ABC transporters\n            ko02020  Two-component system\n            ko05152  Tuberculosis\nMODULE      M00222  Phosphate transport system\nBRITE       KEGG Orthology (KO) [BR:ko00001]\n             Environmental Information Processing\n              Membrane transport\n               02010 ABC transporters\n                K02040  pstS; phosphate transport system substrate-binding protein\n              Signal transduction\n               02020 Two-component system\n                K02040  pstS; phosphate transport system substrate-binding protein\n             Human Diseases\n              Infectious diseases\n               05152 Tuberculosis\n                K02040  pstS; phosphate transport system substrate-binding protein\n            KEGG modules [BR:ko00002]\n             Structural complex\n              Environmental informa

In [15]:
count_vect = CountVectorizer(min_df = 5, analyzer = "word", stop_words="english", ngram_range = (1,2))
X_train_counts = count_vect.fit_transform(kegg_lists)

In [16]:
len(count_vect.vocabulary_)

34814

In [ ]:
X_train_counts.shape

In [ ]:

tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)

In [ ]:
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

In [ ]:
df = pd.read_csv("../data/keggs_RF_conf_scores.csv")
df.head()

In [ ]:
kegg_labels = []
bad_keggs = []
for i, kegg in enumerate(keggs):
    if kegg in list(df.kegg):
        kegg_score = df[df.kegg== kegg]["over"].item()
        if kegg_score > 0.75: 
            kegg_labels.append("over")
        elif kegg_score < 0.25:
            kegg_labels.append("under")
        else:
            kegg_labels.append("neither")
    else:
        kegg_labels.append("unknown")
        bad_keggs.append(i)

In [ ]:
len(kegg_labels)

In [ ]:
len(bad_keggs)

In [ ]:
le = preprocessing.LabelEncoder()
y_labels = le.fit_transform(kegg_labels)

In [ ]:
counter=collections.Counter(y_labels)
counter

In [ ]:
counter=collections.Counter(kegg_labels)
counter

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_counts, y_labels)
predicted = clf.predict(X_train_counts)
mean(predicted == y_labels)

In [ ]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss='log', penalty='l2',alpha=1e-1, n_iter=100, random_state=42)
clf.fit(X_train_counts, y_labels)
predicted = clf.predict(X_train_counts)
mean(predicted == y_labels)

In [ ]:
from sklearn import metrics

In [ ]:
print(metrics.classification_report(y_labels, predicted))

In [ ]:
from sklearn import cross_validation
from sklearn.pipeline import Pipeline

In [ ]:
text_clf = Pipeline([('vect', CountVectorizer(min_df = 5, analyzer = "word", stop_words="english")),
                     ('tfidf', TfidfTransformer(use_idf=False)),
                     ('clf', SGDClassifier(loss='log', penalty='l2',
                                           alpha=1e-5, n_iter=100, random_state=42))])

In [ ]:
parameters = {'tfidf__use_idf': (True, False),
              'clf__alpha': (1e-1, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7)}

In [ ]:
scores = cross_validation.cross_val_score(text_clf, kegg_lists, y_labels, cv = 10, scoring="f1_weighted", n_jobs=-1)


In [ ]:
print mean(scores)
print sqrt(var(scores))

In [ ]:
good_kegg_list = []
for i, kegg in enumerate(keggs):
    if i in bad_keggs:
        continue
    else:
        print 'hi'
        good_kegg_list.append(kegg_queries[kegg])

In [ ]:
scores = cross_validation.cross_val_score(text_clf, good_kegg_list, delete(y_labels, bad_keggs), cv = 10, scoring="f1_weighted", n_jobs=-1)

In [ ]:
print mean(scores)
print sqrt(var(scores))

In [ ]:
gs_clf = GridSearchCV(text_clf, parameters, cv=10, n_jobs=-1, scoring = "f1_weighted")

In [ ]:
gs_clf.fit(good_kegg_list, delete(y_labels, bad_keggs))

In [ ]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(good_kegg_list, delete(y_labels, bad_keggs), test_size=0.2, random_state=0)